In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [60]:
import zipfile
import glob
import os
import PIL
import cv2
from PIL import Image
import shutil
from tqdm import tqdm
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import matplotlib.pyplot as plt

In [3]:
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip',"r") as train_bkp:
    train_bkp.extractall()

with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip',"r") as test:
    test.extractall()

In [4]:
train_path = glob.glob("./train/*")
print(len(train_path))

test_path = glob.glob("./test1/*")
print(len(test_path))

25000
12500


In [5]:
target_list = []
for sample in train_path:
    target = os.path.basename(os.path.normpath(sample)).split(".")[0]
    target_list.append(target)

In [6]:
os.makedirs("./data_train")
os.makedirs("./data_train/cat")
os.makedirs("./data_train/dog")

In [7]:
dogs = 0
cats = 0
source ="./train_bkp"
dog_folder ="./data_train/dog"
cat_folder = "./data_train/cat"

for f,i in zip(train_path, target_list):
    img= Image.open(f)
    img = np.array(img)
    im = cv2.resize(img,(150,150),interpolation=cv2.INTER_LINEAR)
    rescaled_img = (np.maximum(im,0)/im.max())*255
    fin_img = np.uint8(rescaled_img)
    fin_img = Image.fromarray(fin_img)
    name = i
    fin_img.save(f)
    new_path =f
    if name=='dog':
        src_path = new_path
        dst_path = dog_folder
        shutil.move(src_path, dst_path)
        dogs+=1
    else:
        src_path = new_path
        dst_path = cat_folder
        shutil.move(src_path, dst_path)
        cats+=1

In [8]:
head = './data_train'
samples, targets = [], []
for n, categorie in enumerate(['cat', 'dog']):
     files = [f'{head}/{categorie}/{x}' for x in os.listdir(f'{head}/{categorie}')]
     print(f'{categorie}s: {len(files)}')
     samples += files
     targets += [n for i in files]

cats: 12500
dogs: 12500


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
class CustomDataset(Dataset):

    def __init__(
        self,
        image_paths: list,
        targets: list,
        transform: bool):
        """
        Parameters
        ----------
        image_paths : list
            List containing paths to images.
        targets: list
            List containing targets.
        transform: bool
            If true, perform set of transformations to input data.
        """
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        """Return total number of samples in the dataset."""
        return len(self.image_paths)
    
    def __getitem__(self, item):
        """For a given item, returns data corresponding to that index.
        
        Parameters
        ----------

        """
        image = Image.open(self.image_paths[item])
        # image = cv2.imread(self.image_paths[item])
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # augmented = self.augmentations(image=image)
        # image = np.transpose(image, (2,0,1)).astype(float)

        transforms = T.Compose([T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        image = transforms(image)

        target = torch.tensor(self.targets[item])
        
        return {
            'image': image,
            'target': target
        }

In [11]:
class KittenModel(nn.Module):

    def __init__(self):
        super(KittenModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 128, 3)
        
        self.max_pooling = nn.MaxPool2d((2,2))
        self.fc1 = nn.Linear(6272, 512)
        self.fc2 = nn.Linear(512, 2)
        self.classifier = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.max_pooling(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pooling(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.max_pooling(x)

        x = self.conv4(x)
        x = F.relu(x)
        x = self.max_pooling(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        X = F.relu(x)
        x = self.fc2(x)
        x = self.classifier(x)
            
        return x

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## First model
class scratch_nn(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=100, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv2d(100, 200, 3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(200, 400, 3, stride=1, padding=0)
        self.mpool = nn.MaxPool2d(kernel_size=3)
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(6400,1024)
        self.linear2 = nn.Linear(1024,512)
        self.linear3 = nn.Linear(512,2)
        self.classifier = nn.Softmax(dim=1)
        
    def forward(self,x):
        x = self.mpool( self.relu(self.conv1(x)) )
        x = self.mpool( self.relu(self.conv2(x)) )
        x = self.mpool( self.relu(self.conv3(x)) )
        x = torch.flatten(x, start_dim=1)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.classifier(x)
        return x

In [64]:
class Optimization:

    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
        self.val_accuracy = []

    def train_step(self, x, y):
        
        # Set model to train.
        self.model.train()
        # Predict.
        yhat = self.model(x)
        # Compute loss.
        loss = self.loss_fn(yhat, y)
        # Backward pass.
        loss.backward()
        # Updates parameters.
        self.optimizer.step()
        self.optimizer.zero_grad()
        return loss.item()

    def train(self, train_loader, val_loader, n_epochs):
        for epoch in range(n_epochs):
            batch_losses = []
            for batch in tqdm(train_loader):
                x, y = batch.values()
                x = x.to(device)
                y = y.to(device)
                loss = self.train_step(x, y)
                batch_losses.append(loss)
            epoch_loss = np.mean(batch_losses)
            self.train_losses.append(epoch_loss)

            with torch.no_grad():
                self.model.eval()
                batch_losses_val = []
                batch_acc_val = []
                for batch in tqdm(val_loader):
                    x_val, y_val = batch.values()
                    x_val = x_val.to(device)
                    y_val = y_val.to(device)
                    yhat_val = self.model(x_val)
                    loss_val = self.loss_fn(yhat_val, y_val)
                    
                    y_hat_1d = torch.argmax(yhat_val, dim=1)
                    acc_val = accuracy_score([int(i) for i in yhat_val[:,1].cpu().numpy()], y_val.cpu().numpy())
                    
                    batch_losses_val.append(loss_val.item())
                    batch_acc_val.append(acc_val)
                    
                epoch_loss_val = np.mean(batch_losses_val)
                epoch_acc_val = np.mean(batch_acc_val)
                self.val_losses.append(epoch_loss_val)
                self.val_accuracy.append(epoch_acc_val)

            # if epoch % 50 == 0:
            print(
                f'{epoch}/{n_epochs} - training_loss: {epoch_loss:.4f}\t validation_loss: {epoch_loss_val:.4f}\t validation_accuracy: {epoch_acc_val:.4f}')
                
    def plot_losses(self):
        plt.plot(self.train_losses, label="Training loss")
        plt.plot(self.val_losses, label="Validation loss")
        plt.plot(self.val_accuracy, label="Validation accuracy")       
        plt.legend()
        plt.title("Losses")
        plt.show()
        plt.close()

In [ ]:
n_epochs = 500
learning_rate = 1e-3
weight_decay = 1e-6

X_train, X_test, y_train, y_test = train_test_split(samples, targets)

data_transform = T.Compose([
    T.ToTensor(),
    T.Resize((256, 256)),
    T.ColorJitter(),
    T.RandomCrop(224),
    T.RandomHorizontalFlip()
])

train_data = CustomDataset(X_train, y_train, data_transform)
val_data = CustomDataset(X_test, y_test, None)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=True)

model = KittenModel().to(device)
# model = scratch_nn().to(device)
# loss_fn = nn.BCELoss()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=0.9)

opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)

opt.train(train_loader, val_loader, n_epochs)
opt.plot_losses()

100%|██████████| 49/49 [00:09<00:00,  5.15it/s]


0/500 - training_loss: 0.8131	 validation_loss: 0.8119	 validation_accuracy: 0.5014


 35%|███▍      | 51/147 [00:12<00:24,  3.97it/s]